In [1]:
import pandas as pd
import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import pymongo
from pymongo import MongoClient

In [2]:
# Load in our parameters file
with open('../parameters.json') as f:
    params = json.load(f)

In [3]:
# Get relevant parameters
database = params['database']
labeled_collection = params['labeled_collection']
chunk_collection = params['chunk_collection']
six_chunk_collection = params['chunk_6_collection']

# Connect to our database/collection
client = MongoClient('localhost', 27017)
db = client[database]
six_chunk_coll = db[six_chunk_collection]

In [4]:
chunk_df = pd.DataFrame(list(six_chunk_coll.find({},{'_id':0})))
chunk_df.head()

,avg_speed_chnk_1,avg_speed_chnk_2,avg_speed_chnk_3,avg_speed_chnk_4,avg_speed_chnk_5,avg_speed_chnk_6,mfn_sq_chnk_1,mfn_sq_chnk_2,mfn_sq_chnk_3,mfn_sq_chnk_4,mfn_sq_chnk_5,mfn_sq_chnk_6,seconds_chnk_1,seconds_chnk_2,seconds_chnk_3,seconds_chnk_4,seconds_chnk_5,seconds_chnk_6,start_timestamp,trip_id_iso
0,5.782818,5.208333,5.694500,4.722200,5.158857,5.347313,481636,470596,462400,452929,446224,440896,416.0,350.0,440.0,270.0,243.0,481.0,1.475912e+09,7253717_2016-10-08_4489K
1,4.974773,5.540111,4.678316,4.907333,4.120083,5.711080,12321,10609,9025,7396,6400,5184,450.0,424.0,480.0,352.0,410.0,576.0,1.475947e+09,7253845_2016-10-08_GHNYG
2,5.145619,4.204591,5.277826,4.314800,2.800833,3.485710,729,1296,2025,2916,3721,4489,508.0,536.0,454.0,398.0,346.0,1020.0,1.475955e+09,7253837_2016-10-08_8Q6DP
3,6.510500,3.465524,4.514000,2.896810,2.648200,5.785130,28900,31684,35721,39204,43264,46225,464.0,640.0,540.0,524.0,404.0,540.0,1.475963e+09,7253830_2016-10-08_SR1KP
4,4.621318,4.507500,4.638850,2.806895,2.991462,5.611150,96100,102400,108900,114244,121104,125316,568.0,598.0,468.0,544.0,350.0,596.0,1.475972e+09,7253823_2016-10-08_TDW9M


In [5]:
sort_df = chunk_df.sort_values('start_timestamp')
sort_df.reset_index(drop=True, inplace=True)

data_with_priors = []

for idx, row in sort_df.iterrows():

    if idx != 0:
        
        data_dict = {}
        
        prior = sort_df.iloc[idx-1]
        
        current_ts = row['start_timestamp']
        prior_ts = prior['start_timestamp']
        
        diff =  current_ts - prior_ts
        
        if diff < 1800:
            
            data_dict['seconds_chnk_3'] = row['seconds_chnk_3']
            data_dict['seconds_chnk_2'] = row['seconds_chnk_2']
            data_dict['mfn_sq_chnk_2'] = row['mfn_sq_chnk_2']
            data_dict['prior_seconds_chnk_3'] = prior['seconds_chnk_3']
            
            data_with_priors.append(data_dict)

In [6]:
prior_df = pd.DataFrame(data_with_priors)

mask = (prior_df['seconds_chnk_2'] < 1000) & (prior_df['seconds_chnk_3'] < 1000)
prior_trimed_df = prior_df[mask]

y = prior_trimed_df['seconds_chnk_3'].values.reshape(-1,1)
X = prior_trimed_df[['seconds_chnk_2', 'mfn_sq_chnk_2', 'prior_seconds_chnk_3']].values

scaler = StandardScaler()
X_norm = scaler.fit_transform(X)
scaler_y = StandardScaler()
y_norm = scaler_y.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm)

In [7]:
# Build our model
ridge_cv = RidgeCV(alphas=[5,10,20,30], cv=20)
ridge_cv.fit(X_train, y_train)

#Score our model, R-Squared
r_squared = ridge_cv.score(X_test, y_test)

#Score our model, Root Mean Squared Error
y_predict = ridge_cv.predict(X_test)
rmse_norm = (mean_squared_error(y_test, y_predict))**.5
rmse_dev = scaler_y.inverse_transform([rmse_norm])
diff = rmse_dev[0] - scaler_y.mean_[0]

print ("R-squared: ", "{0:.2f}".format(r_squared))
print ('RMSE: ', "{0:.2f}".format(diff))

R-squared:  0.17
RMSE:  63.43


In [8]:
regr = RandomForestRegressor(n_estimators=50, max_depth=2)

crossed = cross_validate(regr, X_train, y_train.flatten(), cv=20, 
                         scoring=['neg_mean_squared_error','r2'], return_train_score=False)

r_squared = crossed['test_neg_mean_squared_error'].mean()

#Score our model, Root Mean Squared Error
rmse_norm = (abs(crossed['test_neg_mean_squared_error'].mean()))**.5
rmse_dev = scaler_y.inverse_transform([rmse_norm])
diff = rmse_dev[0] - scaler_y.mean_[0]

print ("R-squared: ", "{0:.2f}".format(r_squared))
print ('RMSE: ', "{0:.2f}".format(diff))

R-squared:  -0.86
RMSE:  64.76
